In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import math
import re
import time
import gc

In [2]:
start_time = time.time()
df1 = pd.read_csv('NetflixPrizeData/combined_data_1_v2.csv', header = None, names = ['CustomerID', 'Rating','MovieID'], usecols = [0,1,3])
df2 = pd.read_csv('NetflixPrizeData/combined_data_2_v2.csv', header = None, names = ['CustomerID', 'Rating','MovieID'], usecols = [0,1,3])
df3 = pd.read_csv('NetflixPrizeData/combined_data_3_v2.csv', header = None, names = ['CustomerID', 'Rating','MovieID'], usecols = [0,1,3])
df4 = pd.read_csv('NetflixPrizeData/combined_data_4_v2.csv', header = None, names = ['CustomerID', 'Rating','MovieID'], usecols = [0,1,3])

df1['Rating'] = df1['Rating'].astype(float)
df2['Rating'] = df2['Rating'].astype(float)
df3['Rating'] = df3['Rating'].astype(float)
df4['Rating'] = df4['Rating'].astype(float)

print('Dataset 1 shape: {}'.format(df1.shape))
print('-Dataset examples-')
print(df1.iloc[::5000000, :])

print('Time taken:'+str((time.time() - start_time))+' seconds')  

Dataset 1 shape: (24053764, 3)
-Dataset examples-
          CustomerID  Rating  MovieID
0            1488844     3.0        1
5000000       501954     2.0      996
10000000      404654     5.0     1962
15000000      886608     2.0     2876
20000000     1193835     2.0     3825
Time taken:29.71516704559326 seconds


In [16]:
# load less data for speed

df = df1
# df = df1.append(df2)
# df = df.append(df3)
# df = df.append(df4)

df.index = np.arange(0,len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

Full dataset shape: (24053764, 3)
-Dataset examples-
          CustomerID  Rating  MovieID
0            1488844     3.0        1
5000000       501954     2.0      996
10000000      404654     5.0     1962
15000000      886608     2.0     2876
20000000     1193835     2.0     3825


In [17]:
f = ['count','mean']

df_movie_summary = df.groupby('MovieID')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = df.groupby('CustomerID')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

Movie minimum times of review: 1799.0
Customer minimum times of review: 52.0


In [18]:
print('Original Shape: {}'.format(df.shape))
df = df[~df['MovieID'].isin(drop_movie_list)]
df = df[~df['CustomerID'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(df.shape))
print('-Data Examples-')
print(df.iloc[::5000000, :])

Original Shape: (24053764, 3)
After Trim Shape: (17337458, 3)
-Data Examples-
          CustomerID  Rating  MovieID
693           712664     5.0        3
6931106      1299309     5.0     1384
13857613      400155     3.0     2660
20762607      466962     4.0     3923


In [83]:
# First 50000 rows
first_50_df = df[0:500000]
first_50_df

,CustomerID,Rating,MovieID
693,712664,5.0,3
694,1331154,4.0,3
695,2632461,3.0,3
696,44937,5.0,3
697,656399,4.0,3
...,...,...,...
697401,13938,3.0,187
697402,202572,3.0,187
697403,2200010,4.0,187
697405,2583202,3.0,187


In [84]:
duplicates = first_50_df.duplicated(subset='CustomerID', keep=False)
duplicates

693       True
694       True
695       True
696       True
697       True
          ... 
697401    True
697402    True
697403    True
697405    True
697406    True
Length: 500000, dtype: bool

In [21]:
d_np = duplicates.to_numpy()
d_np

array([ True,  True, False, ..., False, False, False])

In [22]:
np.where(d_np == False)

(array([    2,     3,     4, ..., 49997, 49998, 49999]),)

In [85]:
first_50_df.dtypes

CustomerID      int64
Rating        float64
MovieID         int64
dtype: object

In [86]:
x = first_50_df.groupby('Rating')['Rating'].agg(['count'])
print(x)
print(x.sum(axis = 0, skipna = True))

         count
Rating        
1.0      21786
2.0      50347
3.0     145809
4.0     175638
5.0     106420
count    500000
dtype: int64


In [ ]:
df.set_index('CustomerID',inplace=True, drop=True)
df

In [ ]:
df = df.reset_index()
df

In [33]:
print(len(df['CustomerID'].unique()))
print(len(first_50_df['CustomerID'].unique()))

143458
41795


In [34]:
print(len(df['MovieID'].unique()))
print(len(first_50_df['MovieID'].unique()))

1350
7


In [51]:
df_p = pd.pivot_table(first_50_df,values='Rating',index='MovieID',columns='CustomerID',fill_value=0.0)
df_p

CustomerID,7,201,265,383,424,462,592,695,742,906,...,2648885,2648956,2649050,2649067,2649093,2649100,2649308,2649336,2649378,2649426
MovieID,,,,,,,,,,,,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,1,4,0,...,0,4,0,0,0,0,0,4,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,2,0,0,0,0,4
18,0,0,0,0,4,0,0,0,0,0,...,0,0,3,0,0,0,4,0,0,0
26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3,2,0,0,0,0,0
28,4,4,3,3,0,0,4,0,0,3,...,4,0,0,0,0,5,0,0,3,4


In [52]:
df_dict = df_p.to_dict()
df_dict 

0, 8: 0, 16: 0, 17: 0, 18: 0, 26: 0, 28: 5},
 41701: {3: 0, 8: 0, 16: 0, 17: 0, 18: 4, 26: 0, 28: 0},
 41724: {3: 0, 8: 0, 16: 0, 17: 0, 18: 0, 26: 0, 28: 3},
 41737: {3: 0, 8: 0, 16: 0, 17: 0, 18: 0, 26: 0, 28: 3},
 41840: {3: 0, 8: 0, 16: 0, 17: 0, 18: 5, 26: 0, 28: 0},
 41907: {3: 0, 8: 0, 16: 0, 17: 0, 18: 4, 26: 0, 28: 0},
 41920: {3: 0, 8: 0, 16: 0, 17: 0, 18: 0, 26: 0, 28: 4},
 41946: {3: 0, 8: 0, 16: 0, 17: 0, 18: 0, 26: 1, 28: 0},
 42064: {3: 0, 8: 0, 16: 0, 17: 3, 18: 0, 26: 0, 28: 4},
 42093: {3: 0, 8: 0, 16: 0, 17: 3, 18: 0, 26: 0, 28: 0},
 42099: {3: 0, 8: 0, 16: 0, 17: 3, 18: 0, 26: 0, 28: 0},
 42128: {3: 0, 8: 1, 16: 0, 17: 0, 18: 4, 26: 0, 28: 0},
 42147: {3: 0, 8: 4, 16: 0, 17: 0, 18: 0, 26: 0, 28: 0},
 42222: {3: 0, 8: 2, 16: 0, 17: 0, 18: 0, 26: 0, 28: 0},
 42281: {3: 0, 8: 0, 16: 0, 17: 0, 18: 5, 26: 0, 28: 0},
 42362: {3: 0, 8: 0, 16: 0, 17: 0, 18: 0, 26: 0, 28: 4},
 42416: {3: 0, 8: 1, 16: 0, 17: 0, 18: 0, 26: 0, 28: 0},
 42445: {3: 0, 8: 0, 16: 0, 17: 5, 18: 0, 2

In [54]:
df_title = pd.read_csv('NetflixPrizeData/movie_titles2.csv', encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'],usecols = [0,1,2])
df_title.set_index('Movie_Id', inplace = True)
df_title

,Year,Name
Movie_Id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW
...,...,...
17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17767,2004.0,Fidel Castro: American Experience
17768,2000.0,Epoch


In [81]:
# get column value by index pandas
df_title.loc[1,'Name']

'Dinosaur Planet'

In [78]:
 df_title.columns.tolist()

['Year', 'Name']

In [38]:
movie_titles = df_title.index
movie_titles

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            17761, 17762, 17763, 17764, 17765, 17766, 17767, 17768, 17769,
            17770],
           dtype='int64', name='Movie_Id', length=17770)

In [ ]:
df_p_raw = df_p.copy()

In [ ]:
pickle.dump(df_p,open('pickleFiles/pivotTable.p','wb'))

In [82]:
movie_titles = movie_titles.tolist() #convert pandas.core.indexes.numeric.Int64Index to list type to use with pivot table

AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
movie_ID_list = list(map(str, range(1,17771))) #method 2 create a int list
print(movie_ID_list[:5])
print(movie_ID_list[17765:])

In [ ]:
df.columns.tolist() #extra

In [ ]:
if not df_p:
    df_p = pickle.load(open('pivotTable.p','rb'))

In [40]:
np_array = df_p.to_numpy()
len(np_array)

41795

In [41]:
np_array

array([[0, 5, 0, ..., 0, 0, 4],
       [0, 0, 0, ..., 0, 0, 4],
       [0, 0, 0, ..., 0, 0, 3],
       ...,
       [0, 4, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 4]])

In [ ]:
pickle.dump(np_array,open('pickleFiles/numpyArray.p','wb'))

In [ ]:
del df
gc.collect()